In [75]:
import os, json
import pandas as pd

# this finds our json files
path_to_json = 'F:/kalu/BITS/DeepLearning/assignment1/CORD/train/json/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = pd.DataFrame(columns=['validline'])

f = open("F:/kalu/BITS/DeepLearning/assignment1/receipt_words.csv", "a")
print("filename",",","index",",","x1",",",'x2',",",'x3',",",'x4',",",'y1',",",'y2',",",'y3',",",'y4',",",'is_key',",",'row_id',",",'text',",",'category',",","group_id",file=f)


# we need both the json and an index number so use enumerate()
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)
        
       # print(json_text)

        # here you need to know the layout of your json and each json has to have
        # the same structure (obviously not the structure I have here)
        lines = json_text['valid_line']
        
        for x in lines:
            words = x['words']
            category = x['category']
            group_id = x['group_id']
            for y in words:
                print(js,index,y['quad']['x1'],y['quad']['x2'],y['quad']['x3'],y['quad']['x4'],y['quad']['y1'],y['quad']['y2'],y['quad']['y3'],y['quad']['y4'],y['is_key'],y['row_id'],"\""+y['text']+"\"",category,group_id,file=f,sep=",")

       # city = json_text['features'][0]['properties']['name']
        #lonlat = json_text['features'][0]['geometry']['coordinates']
        # here I push a list of data into a pandas DataFrame at row given by 'index'
        #f.close()
        #jsons_data.loc[index] = [words, city, lonlat]

# now that we have the pertinent json data in our DataFrame let's look at it
f.close()

In [1]:
import pandas as pd
receipts_df = pd.read_csv("F:/kalu/BITS/DeepLearning/assignment1/receipt_words.csv")
receipts_df.columns = receipts_df.columns.str.strip()
receipts_df['text'] = receipts_df['text'].str.strip()


In [2]:
receipts_df.dtypes

filename    object
index        int64
x1           int64
x2           int64
x3           int64
x4           int64
y1           int64
y2           int64
y3           int64
y4           int64
is_key       int64
row_id       int64
text        object
category    object
group_id     int64
dtype: object

In [3]:
char2idx = {"START":0,"END":1,"a":2,"b":3,"c":4,"d":5,"e":6,"f":7,"g":8,"h":9,"i":10,"j":11,"k":12,"l":13,"m":14,"n":15,"o":16,"p":17,"q":18,"r":19,"s":20,"t":21,"u":22,"v":23,"w":24,"x":25,"y":26,"z":27,"0":28,"1":29,"2":30,"3":31,"4":32,"5":33,"6":34,"7":35,"8":36,"9":37,".":38,"-":39,",":40,":":41," ":42}

In [4]:
#p_test.SentimentText=p_test.SentimentText.astype(str)
receipts_df.text = receipts_df.text.astype(str)
receipts_df.dtypes

filename    object
index        int64
x1           int64
x2           int64
x3           int64
x4           int64
y1           int64
y2           int64
y3           int64
y4           int64
is_key       int64
row_id       int64
text        object
category    object
group_id     int64
dtype: object

In [5]:
words = receipts_df['text']

In [6]:
indexed_words = []
j=43
for word in words:
    indexed_word = []
    #print(word)
    word = word.lower();
    for i in range(len(word)):
        token = word[i]
        if token not in char2idx:
            char2idx[token] = j
            j += 1
        indexed_word.append(char2idx[token])
    indexed_words.append(indexed_word)

receipts_df["indexed_words"] = indexed_words;

In [7]:
import numpy as np
def get_bigram_probs(words, V, start_idx, end_idx, smoothing=1):
  # structure of bigram probability matrix will be:
  # (last word, current word) --> probability
  # we will use add-1 smoothing
  # note: we'll always ignore this from the END token
  bigram_probs = np.ones((V, V)) * smoothing
  for word in words:
    for i in range(len(word)):
      
      if i == 0:
        # beginning word
        word[i]
        bigram_probs[start_idx, word[i]] += 1
      else:
        # middle word
        bigram_probs[word[i-1], word[i]] += 1

      # if we're at the final word
      # we update the bigram for last -> current
      # AND current -> END token
      if i == len(word) - 1:
        # final word
        bigram_probs[word[i], end_idx] += 1

  # normalize the counts along the rows to get probabilities
  bigram_probs /= bigram_probs.sum(axis=1, keepdims=True)
  return bigram_probs

In [8]:
start_idx = char2idx['START']
end_idx = char2idx['END']
V = len(char2idx)



In [9]:
bigram_probs = get_bigram_probs(indexed_words, V, start_idx, end_idx, smoothing=0.1)

In [10]:
bigram_probs

array([[5.27301010e-06, 5.27301010e-06, 1.57715732e-02, ...,
        5.27301010e-06, 1.10733212e-04, 1.10733212e-04],
       [1.53846154e-02, 1.53846154e-02, 1.53846154e-02, ...,
        1.53846154e-02, 1.53846154e-02, 1.53846154e-02],
       [1.57915515e-05, 7.97631267e-02, 8.05369128e-04, ...,
        1.57915515e-05, 1.57915515e-05, 1.57915515e-05],
       ...,
       [1.17647059e-02, 1.17647059e-02, 1.17647059e-02, ...,
        1.17647059e-02, 1.17647059e-02, 1.17647059e-02],
       [1.17647059e-02, 2.47058824e-01, 1.17647059e-02, ...,
        1.17647059e-02, 1.17647059e-02, 1.17647059e-02],
       [1.17647059e-02, 1.17647059e-02, 1.29411765e-01, ...,
        1.17647059e-02, 1.17647059e-02, 1.17647059e-02]])

In [11]:
 # a function to calculate normalized log prob score
  # for a sentence
def get_score(word):
    #print("word")
    #print(word)
    score = 0
    if len(word) > 0 :
        for i in range(len(word)):
          if i == 0:
            # beginning word
            score += np.log(bigram_probs[start_idx, word[i]])
          else:
            # middle word
            score += np.log(bigram_probs[word[i-1], word[i]])
        # final word
        score += np.log(bigram_probs[word[i-1], end_idx])

    # normalize the score
    # return score / (len(word) + 1)
    if score == 0:
        return score;
    else :
        return  ((-1)*(len(word) + 1)/score)*100

In [12]:
from future.utils import iteritems

idx2char = dict((v, k) for k, v in iteritems(char2idx))
def get_word(word):
    return ''.join(idx2char[i] for i in word)

In [13]:
# when we sample a fake sentence, we want to ensure not to sample
  # start token or end token
sample_probs = np.ones(V)
sample_probs[start_idx] = 0
sample_probs[end_idx] = 0
sample_probs /= sample_probs.sum()
 # test our model on real and fake sentences
while True:
    # real sentence
    real_idx = np.random.choice(len(indexed_words))
    real = indexed_words[real_idx]
    # fake sentence
    fake = np.random.choice(V, size=len(real), p=sample_probs)

    print("REAL:", get_word(real), "SCORE:", get_score(real))
    print("FAKE:", get_word(fake), "SCORE:", get_score(fake))

    # input your own sentence
    custom = input("Enter your own sentence:\n")
    #custom = custom.lower().split()
    custom
    # check that all tokens exist in word2idx (otherwise, we can't get score)
    bad_sentence = False
    for i in range(len(custom)):
        token = custom[i]
        #print("custom token i")
        #print(token)
        if token not in char2idx:
            bad_sentence = True

    if bad_sentence:
      print("Sorry, you entered words that are not in the vocabulary")
    else:
      # convert sentence into list of indexes
      custom = [char2idx[token] for token in custom]
      print("SCORE:", get_score(custom))


    cont = input("Continue? [Y/n]")
    if cont and cont.lower() in ('N', 'n'):
      break


REAL: ku SCORE: 36.43195142778534
FAKE: *v SCORE: 19.43362014375279


Enter your own sentence:
 


SCORE: 0


Continue? [Y/n] n


In [14]:
receipts_df['text_score'] = receipts_df.apply(lambda row : get_score(row['indexed_words']), axis = 1)

In [15]:
receipts_df

,filename,index,x1,x2,x3,x4,y1,y2,y3,y4,is_key,row_id,text,category,group_id,indexed_words,text_score
0,receipt_00000.json,0,256,270,270,256,374,374,390,390,0,2179893,x,menu.cnt,3,[25],39.091172
1,receipt_00000.json,0,258,270,270,258,402,402,418,418,0,2179894,x,menu.cnt,4,[25],39.091172
2,receipt_00000.json,0,258,272,272,258,428,428,444,444,0,2179895,x,menu.cnt,5,[25],39.091172
3,receipt_00000.json,0,260,274,274,260,456,456,470,470,0,2179896,x,menu.cnt,6,[25],39.091172
4,receipt_00000.json,0,258,274,274,258,480,480,496,496,0,2179897,x,menu.cnt,7,[25],39.091172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18953,receipt_00799.json,786,1002,1179,1180,1002,1465,1465,1531,1528,0,678299,18.000,total.total_price,4,"[29, 36, 38, 28, 28, 28]",73.278820
18954,receipt_00799.json,786,274,415,414,274,1616,1618,1685,1683,1,678300,Tunai,total.cashprice,4,"[21, 22, 15, 2, 10]",35.914549
18955,receipt_00799.json,786,1002,1173,1176,1002,1621,1621,1685,1686,0,678300,19.000,total.cashprice,4,"[29, 37, 38, 28, 28, 28]",64.601556
18956,receipt_00799.json,786,268,474,475,268,1693,1695,1762,1761,1,678301,Kembali,total.changeprice,4,"[12, 6, 14, 3, 2, 13, 10]",49.430316


In [48]:
categories = receipts_df['category'] 
category2idx = {}
idx2category = {}
j=0;
for category in categories:
    if category not in category2idx:
        category2idx[category] = j
        idx2category[j] = category
        j+=1

category2idx

def get_index_for_category(category):
    return category2idx[category]


def get_category_for_index(index):
    return idx2category[index]
        

In [49]:
receipts_df['category_idx'] = receipts_df.apply(lambda row : get_index_for_category(row['category']), axis = 1)

In [50]:
len(category2idx)

30

In [51]:
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras import layers

In [52]:
data = receipts_df[['x1','x2','x3','x4','y1','y2','y3','y4','is_key','text_score','category_idx']]


In [53]:
dnnModel = models.Sequential()

In [54]:
# Layer 1 = input layer
# specify the input size in the first layer.

dnnModel.add(layers.Dense(50, activation='relu', input_shape= (10,)))

# Layer 2 = hidden layer 
dnnModel.add(layers.Dense(60, activation='relu'))

# Layer 3 = hidden layer 
dnnModel.add(layers.Dense(30, activation='relu'))

# Layer 4 = output layer
#dnnModel.add(layers.Dense(len(category2idx), activation='softmax'))

dnnModel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 50)                550       
_________________________________________________________________
dense_7 (Dense)              (None, 60)                3060      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 61        
Total params: 3,671
Trainable params: 3,671
Non-trainable params: 0
_________________________________________________________________


In [55]:
dnnModel.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'] )

In [56]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)

In [57]:
y = data.pop('category_idx')


In [58]:
y

0         0
1         0
2         0
3         0
4         0
         ..
18953     7
18954     9
18955     9
18956    10
18957    10
Name: category_idx, Length: 18958, dtype: int64

In [59]:
X = data



In [60]:
X

,x1,x2,x3,x4,y1,y2,y3,y4,is_key,text_score
0,256,270,270,256,374,374,390,390,0,39.091172
1,258,270,270,258,402,402,418,418,0,39.091172
2,258,272,272,258,428,428,444,444,0,39.091172
3,260,274,274,260,456,456,470,470,0,39.091172
4,258,274,274,258,480,480,496,496,0,39.091172
...,...,...,...,...,...,...,...,...,...,...
18953,1002,1179,1180,1002,1465,1465,1531,1528,0,73.278820
18954,274,415,414,274,1616,1618,1685,1683,1,35.914549
18955,1002,1173,1176,1002,1621,1621,1685,1686,0,64.601556
18956,268,474,475,268,1693,1695,1762,1761,1,49.430316


In [61]:
X_train,X_test,y_train,y_test = train_test_split(X.index,y.index,test_size=0.2)

In [62]:
xtrain=X.iloc[X_train]

In [63]:
xtest=X.iloc[X_test]

In [64]:
y_train=y.iloc[y_train]

In [65]:
y_test=y.iloc[y_train]

In [66]:
unique_values = pd.unique(y_train)
print(unique_values)
unique_values = pd.unique(y_test)
print(unique_values)

[ 0  9 10 21  1 12 14  6 18  8  4  2 15  3  7  5 19 13 20 17 16 11 26 23
 22 27 25 29 24 28]
[0 2 1]


In [67]:
y_train.shape
type(y_train)

y_train = y_train.to_frame('target')


In [68]:
y_train.dtypes
y_train.shape

(15166, 1)

In [491]:
def prepare_inputs(X_train, X_test):
	ohe = OneHotEncoder(handle_unknown = 'ignore')
	ohe.fit(X_train)
	X_train_enc = ohe.transform(X_train)
	X_test_enc = ohe.transform(X_test)
	return X_train_enc, X_test_enc

In [492]:
y_train = y_train.to_numpy().reshape((len(y_train), 1))
y_test = y_test.to_numpy().reshape((len(y_test), 1))
y_train, y_test = prepare_inputs(y_train,y_test)
y_test=pd.DataFrame(y_test)
y_train=pd.DataFrame(y_train)




In [69]:
y_train.shape
y_train.dtypes

target    int64
dtype: object

In [70]:
y_train.shape

(15166, 1)

In [71]:
y_train

,target
18317,0
13769,9
12915,10
6823,21
8462,10
...,...
10725,3
17509,16
7861,7
18001,0


In [72]:
y_train.shape

(15166, 1)

In [73]:
xtrain=tf.convert_to_tensor(xtrain)

y_train=tf.convert_to_tensor(y_train)

In [74]:
y_train.shape

TensorShape([15166, 1])

In [75]:
xtrain.shape

TensorShape([15166, 10])

In [76]:
h  = dnnModel.fit( xtrain, y_train, epochs=25, batch_size=64)

Epoch 1/25
237/237 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.1010
Epoch 2/25
237/237 [==============================] - 0s 974us/step - loss: nan - accuracy: 0.1009
Epoch 3/25
237/237 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.1009
Epoch 4/25
237/237 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.1009
Epoch 5/25
237/237 [==============================] - 0s 975us/step - loss: nan - accuracy: 0.1009
Epoch 6/25
237/237 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.1009
Epoch 7/25
237/237 [==============================] - 0s 949us/step - loss: nan - accuracy: 0.1009
Epoch 8/25
237/237 [==============================] - 0s 984us/step - loss: nan - accuracy: 0.1009
Epoch 9/25
237/237 [==============================] - 0s 924us/step - loss: nan - accuracy: 0.1009
Epoch 10/25
237/237 [==============================] - 0s 970us/step - loss: nan - accuracy: 0.1009
Epoch 11/25
237/2

In [36]:
xtrain.dtypes

x1             object
x2             object
x3             object
x4             object
y1             object
y2             object
y3             object
y4             object
is_key         object
text_score    float64
dtype: object

In [38]:
df_new = xtrain.infer_objects()

In [39]:
df_new.dtypes

x1             object
x2             object
x3             object
x4             object
y1             object
y2             object
y3             object
y4             object
is_key         object
text_score    float64
dtype: object

In [109]:
df_new['x1'].str.strip().astype('int')

ValueError: invalid literal for int() with base 10: 'x1'

In [44]:
df_new['x1'] =   df_new['x1'].str.strip()

In [45]:
df_new.dtypes

x1             object
x2             object
x3             object
x4             object
y1             object
y2             object
y3             object
y4             object
is_key         object
text_score    float64
dtype: object

In [46]:
df_new['x1'] = pd.to_numeric(df_new['x1'])

ValueError: Unable to parse string "x1" at position 1907

In [48]:
df_new['x1'][1908]

KeyError: 1908

In [49]:
len(df_new['x1'])

30336

In [50]:
df_new['x1'][1906]

KeyError: 1906

In [53]:
df_new['x1'][1907]

'58'

In [54]:
df_new

,x1,x2,x3,x4,y1,y2,y3,y4,is_key,text_score
7541,121,142,143,122,826,826,864,864,0,75.727517
16723,246,310,310,246,914,914,952,952,0,40.133252
3208,182,260,260,182,428,428,452,452,0,40.210847
22685,260,278,278,260,624,624,646,646,0,40.957955
16450,123,239,242,127,1048,1037,1074,1085,1,60.661059
...,...,...,...,...,...,...,...,...,...,...
23213,724,849,851,726,1204,1198,1244,1249,0,61.194116
3575,474,576,576,474,748,748,784,784,0,65.125374
28875,464,538,538,464,812,812,856,856,0,41.688069
20974,64,194,193,63,887,890,923,921,1,66.044601


In [55]:
a=10
a

10

In [56]:
a='58'

In [57]:
a

'58'

In [58]:
df_new.dtypes

x1             object
x2             object
x3             object
x4             object
y1             object
y2             object
y3             object
y4             object
is_key         object
text_score    float64
dtype: object

In [62]:
df_new['x1'] = df_new['x1'].astype(str)

In [63]:
df_new['x1'] = df_new['x1'].astype(int)

ValueError: invalid literal for int() with base 10: 'x1'